# <div style="text-align:center"> DS7333 - Case Study 6 | SPAM</div>
### <div style="text-align:center">Andy Ho, An Nguyen, Jodi Pafford</div>
<div style="text-align:center">June 17, 2019</div>

## 1 Introduction

## 2 Background

## 3 Methods

## 4 Results

## 5 Conclusion

### References
+ Nolan, D. and Lang, D. T. “Data Science in R.” CRC Press, 2015 (Chapter 1)
+ http://rdatasciencecases.org/
+ https://rstudio-pubs-static.s3.amazonaws.com/351788_b8d5de284dd645a1b920b7bd77e0967b.html

## 6 Appendix - Code

# Questions to choose from
19. Consider the other parameters that can be used to control the recursive partitioning process. Read the documentation for them in the rpart.control() documentation. Also, carry out an Internet search for more information on how to tweak the rpart() tuning parameters. Experiment with values for these parameters. Do the trees that result make sense with your understanding of how the parameters are used? Can you improve the prediction using them?

20. In the section called “Classifying New Messages” we used the test set that we had put aside to both select , the threshold for the log odds, and to evaluate the Type I and II errors incurred when we use this threshold. Ideally, we choose from another set of messages that is both independent of our training data and our test data. The method of cross-validation is designed to use the training set for training and validating the model. Implement 5-fold cross-validation to choose and assess the error rate with our training data. To do this, follow the steps:

        a. Use the sample() function to permute the indices of the training set, and organize these permuted indices into 5 equal-size sets, called folds.

        b. For each fold, take the corresponding subset from the training data to use as a 'test' set. Use the remaining messages in the training data as the training set. Apply the functions developed in the section called “Implementing the Naïve Bayes Classifier” to estimate the probabilities that a word occurs in a message given it is spam or ham, and use these probabilities to compute the log likelihood ratio for the messages in the training set.

        c. Pool all of the LLR values from the messages in all of the folds, i.e., from all of the training data, and use these values and the typeIErrorRate() function to select a threshold that achieves a 1% Type I error.

        d. Apply this threshold to our original/real test set and find its Type I and Type II errors.

21. Use the sample() function to permute the indices of the training set, and organize these permuted indices into 5 equal-size sets, called folds.

Ok so you can change the dir structure, but this code does depend on data being in certain directories:


`This Notebook`

`    < spam path folder > `

        messages
    
            easy_ham
        
            spam
        
            etc....
        `
`

In [1]:
spamPath = "./SpamAssassinMessages/"
dirNames = list.files(path = paste(spamPath, "messages",  sep = .Platform$file.sep))
fullDirNames = paste(spamPath, "messages", dirNames, sep = .Platform$file.sep)

In [2]:
splitMessage = function(msg) {
  splitPoint = match("", msg)
  header = msg[1:(splitPoint-1)]
  body = msg[ -(1:splitPoint) ]
  return(list(header = header, body = body))
}

getBoundary = function(header) {
  boundaryIdx = grep("boundary=", header)
  boundary = gsub('"', "", header[boundaryIdx])
  gsub(".*boundary= *([^;]*);?.*", "\\1", boundary)
}

dropAttach = function(body, boundary){
  
  bString = paste("--", boundary, sep = "")
  bStringLocs = which(bString == body)
  
  if (length(bStringLocs) <= 1) return(body)
  
  eString = paste("--", boundary, "--", sep = "")
  eStringLoc = which(eString == body)
  if (length(eStringLoc) == 0) 
    return(body[ (bStringLocs[1] + 1) : (bStringLocs[2] - 1)])
  
  n = length(body)
  if (eStringLoc < n) 
     return( body[ c( (bStringLocs[1] + 1) : (bStringLocs[2] - 1), 
                    ( (eStringLoc + 1) : n )) ] )
  
  return( body[ (bStringLocs[1] + 1) : (bStringLocs[2] - 1) ])
}

In [3]:
library(tm)

stopWords = stopwords()
cleanSW = tolower(gsub("[[:punct:]0-9[:blank:]]+", " ", stopWords))
SWords = unlist(strsplit(cleanSW, "[[:blank:]]+"))
SWords = SWords[ nchar(SWords) > 1 ]
stopWords = unique(SWords)

Warning message:
"package 'tm' was built under R version 3.4.4"Loading required package: NLP


In [4]:
cleanText =
function(msg)   {
  tolower(gsub("[[:punct:]0-9[:space:][:blank:]]+", " ", msg))
}

findMsgWords = 
function(msg, stopWords) {
 if(is.null(msg))
  return(character())

 words = unique(unlist(strsplit(cleanText(msg), "[[:blank:]\t]+")))
 
 # drop empty and 1 letter words
 words = words[ nchar(words) > 1]
 words = words[ !( words %in% stopWords) ]
 invisible(words)
}

In [5]:
processAllWords = function(dirName, stopWords)
{
       # read all files in the directory
  fileNames = list.files(dirName, full.names = TRUE)
       # drop files that are not email, i.e., cmds
  notEmail = grep("cmds$", fileNames)
  if ( length(notEmail) > 0) fileNames = fileNames[ - notEmail ]

  messages = lapply(fileNames, readLines, encoding = "latin1")
  
       # split header and body
  emailSplit = lapply(messages, splitMessage)
       # put body and header in own lists
  bodyList = lapply(emailSplit, function(msg) msg$body)
  headerList = lapply(emailSplit, function(msg) msg$header)
  rm(emailSplit)
  
       # determine which messages have attachments
  hasAttach = sapply(headerList, function(header) {
    CTloc = grep("Content-Type", header)
    if (length(CTloc) == 0) return(0)
    multi = grep("multi", tolower(header[CTloc])) 
    if (length(multi) == 0) return(0)
    multi
  })
  
  hasAttach = which(hasAttach > 0)
  
       # find boundary strings for messages with attachments
  boundaries = sapply(headerList[hasAttach], getBoundary)
  
       # drop attachments from message body
  bodyList[hasAttach] = mapply(dropAttach, bodyList[hasAttach], 
                               boundaries, SIMPLIFY = FALSE)
  
       # extract words from body
  msgWordsList = lapply(bodyList, findMsgWords, stopWords)
  
  invisible(msgWordsList)
}

In [6]:
msgWordsList = lapply(fullDirNames, processAllWords, 
                      stopWords = stopWords) 

Warning message in FUN(X[[i]], ...):
"incomplete final line found on './SpamAssassinMessages//messages/spam/00136.faa39d8e816c70f23b4bb8758d8a74f0'"Warning message in FUN(X[[i]], ...):
"incomplete final line found on './SpamAssassinMessages//messages/spam/0143.260a940290dcb61f9327b224a368d4af'"

In [7]:
numMsgs = sapply(msgWordsList, length)
numMsgs

[1] 5051 1400  500 1000 1397

In [8]:
isSpam = rep(c(FALSE, FALSE, FALSE, TRUE, TRUE), numMsgs)

msgWordsList = unlist(msgWordsList, recursive = FALSE)

numEmail = length(isSpam)
numSpam = sum(isSpam)
numHam = numEmail - numSpam

set.seed(418910)

#number of folds to produce
fold = 5

#initialize lists of indexes
foldSpamIdx = list()
foldHamIdx = list()

#create a list of x folds holding a list of indexes
#sample() will not sample indexes that have already been taken for previous folds
for (x in 1:fold){
    foldSpamIdx[x] = list(sample((1:numSpam)[!((1:numSpam) %in% unlist(foldSpamIdx))], size = floor(numSpam/fold)))
    foldHamIdx[x] = list(sample((1:numHam)[!((1:numHam) %in% unlist(foldHamIdx))], size = floor(numHam/fold)))
}

In [9]:
computeFreqs =
function(wordsList, spam, bow = unique(unlist(wordsList)))
{
   # create a matrix for spam, ham, and log odds
  wordTable = matrix(0.5, nrow = 4, ncol = length(bow), 
                     dimnames = list(c("spam", "ham", 
                                        "presentLogOdds", 
                                        "absentLogOdds"),  bow))

   # For each spam message, add 1 to counts for words in message
  counts.spam = table(unlist(lapply(wordsList[spam], unique)))
  wordTable["spam", names(counts.spam)] = counts.spam + .5

   # Similarly for ham messages
  counts.ham = table(unlist(lapply(wordsList[!spam], unique)))  
  wordTable["ham", names(counts.ham)] = counts.ham + .5  


   # Find the total number of spam and ham
  numSpam = sum(spam)
  numHam = length(spam) - numSpam

   # Prob(word|spam) and Prob(word | ham)
  wordTable["spam", ] = wordTable["spam", ]/(numSpam + .5)
  wordTable["ham", ] = wordTable["ham", ]/(numHam + .5)
  
   # log odds
  wordTable["presentLogOdds", ] = 
     log(wordTable["spam",]) - log(wordTable["ham", ])
  wordTable["absentLogOdds", ] = 
     log((1 - wordTable["spam", ])) - log((1 -wordTable["ham", ]))

  invisible(wordTable)
}

In [10]:
#create a list of word table, 1 for each fold as the test set
trainTable = list()
testMsgWords = list()
testIsSpam = list()
trainIsSpam = list()

for (x in 1:fold){
    testSpamIdx = foldSpamIdx[[x]]
    testHamIdx = foldHamIdx[[x]]

    testMsgWords[[x]] = c((msgWordsList[isSpam])[testSpamIdx],
                     (msgWordsList[!isSpam])[testHamIdx] )
    trainMsgWords = c((msgWordsList[isSpam])[ - testSpamIdx], 
                      (msgWordsList[!isSpam])[ - testHamIdx])

    testIsSpam[[x]] = rep(c(TRUE, FALSE), 
                     c(length(testSpamIdx), length(testHamIdx)))
    trainIsSpam[[x]] = rep(c(TRUE, FALSE), 
                     c(numSpam - length(testSpamIdx), 
                       numHam - length(testHamIdx)))

    trainTable[[x]] = list(computeFreqs(trainMsgWords, trainIsSpam[[x]]))
}

In [11]:
computeMsgLLR = function(words, freqTable) 
{
       # Discards words not in training data.
  words = words[!is.na(match(words, colnames(freqTable)))]

       # Find which words are present
  present = colnames(freqTable) %in% words

  sum(freqTable["presentLogOdds", present]) +
    sum(freqTable["absentLogOdds", !present])
}

In [12]:
testLLR = list()
for (x in 1:fold){
    testLLR[[x]] = sapply(testMsgWords[[x]], computeMsgLLR, trainTable[[x]][[1]])
}

[1] "Fold 1:"
$`FALSE`
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-844.88 -126.92  -98.05 -116.89  -77.28  746.25 

$`TRUE`
    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
  -56.55    13.64    58.65   179.92   146.84 23524.91 

[1] "Fold 2:"
$`FALSE`
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-866.19 -135.78 -108.61 -125.88  -86.79   95.79 

$`TRUE`
    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
 -75.653    2.039   48.668  101.734  128.926 1446.490 

[1] "Fold 3:"
$`FALSE`
    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
-1099.20  -134.40  -105.34  -123.34   -83.98    14.48 

$`TRUE`
     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
  -73.267     2.469    56.071   146.890   122.165 23516.206 

[1] "Fold 4:"
$`FALSE`
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-696.79 -132.56 -105.14 -122.86  -85.08  107.62 

$`TRUE`
    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
 -65.873    7.451   55.138  119.337  134.529 3450.687 

[1] "Fold 5:

In [21]:
typeIErrorRates = 
function(llrVals, isSpam) 
{
  o = order(llrVals)
  llrVals =  llrVals[o]
  isSpam = isSpam[o]

  idx = which(!isSpam)
  N = length(idx)
  list(error = (N:1)/N, values = llrVals[idx])
}

In [30]:
xI = typeIErrorRates(testLLR[[1]], testIsSpam[[1]])
tau01 = round(min(xI$values[xI$error <= 0.01]))
tau01

[1] -6